## Preparing the Data

In [ ]:
import os
import cv2
import torch
import logging
from telethon import TelegramClient

# Configure logging
logging.basicConfig(filename='yolo_detection.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Use your own values from my.telegram.org
api_id = '28822767'
api_hash = '6b5b1a99af22f4c184d5a7acd8a7c27f'

# Create the client and connect
client = TelegramClient('session_name', api_id, api_hash)

# Load YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

async def download_images(channel, limit=10):
    images = []
    async for message in client.iter_messages(channel, limit=limit):
        if message.photo:
            photo = await message.download_media()
            images.append(photo)
    return images

async def main():
    channels = ['https://t.me/lobelia4cosmetics']
    for channel in channels:
        images = await download_images(channel)
        for image_path in images:
            img = cv2.imread(image_path)
            results = model(img)
            results.save()  # Save results to disk
            logging.info(f"Processed image: {image_path}")

with client:
    client.loop.run_until_complete(main())

: 

## Processing the Detection Results

In [ ]:
import sqlite3

# Create database connection
conn = sqlite3.connect('detection_data.db')
c = conn.cursor()

# Create table
c.execute('''
CREATE TABLE IF NOT EXISTS detections (
    id INTEGER PRIMARY KEY,
    image_path TEXT,
    class_label TEXT,
    confidence REAL,
    bbox TEXT
)
''')

async def main():
    channels = ['https://t.me/lobelia4cosmetics']
    for channel in channels:
        images = await download_images(channel)
        for image_path in images:
            img = cv2.imread(image_path)
            results = model(img)
            for *box, conf, cls in results.xyxy[0]:
                bbox = ','.join(map(str, box))
                class_label = model.names[int(cls)]
                c.execute('''
                INSERT INTO detections (image_path, class_label, confidence, bbox)
                VALUES (?, ?, ?, ?)
                ''', (image_path, class_label, conf.item(), bbox))
                conn.commit()
            logging.info(f"Processed image: {image_path}")

with client:
    client.loop.run_until_complete(main())

conn.close()